In [2]:
import config, data_load
import pandas as pd

In [3]:
def read_query(path_query):
  with open(path_query) as f:
    query = f.read()
    df = data_load.df_query(query)
  return df

In [5]:
path_query = 'queries/compare/cnes_sinasc.sql'
read_query(path_query)

,records
0,8191.0


In [4]:
path_query = 'queries/compare/cnes_sih.sql'
read_query(path_query)

,records
0,4176.0


In [17]:
path_query = 'queries/check/cnes_sus.sql'
read_query(path_query)

,count(distinct CNES)
0,214.0


In [3]:
path_query = 'queries/check/sih.sql'
read_query(path_query)

,res_codigo_adotado,int_codigo_adotado,procedimentos
0,330455,330455,56423
1,330170,330170,8974
2,330350,330350,5973
3,330100,330100,4208
4,330490,330490,3293
...,...,...,...
785,330620,330610,1
786,330630,270915,1
787,330630,330411,1
788,330630,330455,1


In [18]:
fname = 'sih_rj_edges_normal_2018_19_diff.csv' # _diff
df = df[df['res_codigo_adotado'] != df['int_codigo_adotado']]
df.to_csv(fname, index=False)

In [24]:
dist = config.PATH_DATABASES + '/DISTANCIAS/matriz_distancias.zip'
dist

'/Users/lucas/Databases/DISTANCIAS/matriz_distancias/matriz_distancias.zip'

In [26]:
df_dist = pd.read_csv(dist)
df_dist

,origem,destino,distancia,tempo
0,110001,110001,0.000000,0.000000
1,110001,110002,309.050000,6.169056
2,110001,110003,399.499700,6.289056
3,110001,110004,81.201103,1.917750
4,110001,110005,391.704300,6.136361
...,...,...,...,...
31024895,530010,522200,161.728900,2.356000
31024896,530010,522205,382.708800,5.479556
31024897,530010,522220,161.603400,2.248944
31024898,530010,522230,190.000900,3.216083


In [3]:
edges_rj = 'data/consult/edges/rj/'

ces_10 = 'sih_rj_edges_cesarea_2010_11_diff.csv'
ces_18 = 'sih_rj_edges_cesarea_2018_19_diff.csv'
nor_10 = 'sih_rj_edges_normal_2010_11_diff.csv'
nor_18 = 'sih_rj_edges_normal_2018_19_diff.csv'

ces_10_all = 'sih_rj_edges_cesarea_2010_11.csv'
ces_18_all = 'sih_rj_edges_cesarea_2018_19.csv'
nor_10_all = 'sih_rj_edges_normal_2010_11.csv'
nor_18_all = 'sih_rj_edges_normal_2018_19.csv'

In [45]:
def modify_df(fname, bienio, tipo_parto):
	df = pd.read_csv(edges_rj + fname)
	df = df.rename(columns={
		'res_codigo_adotado' : 'origem',
		'int_codigo_adotado' : 'destino',
	})
	df['biênio'] = bienio
	df['tipo_parto'] = tipo_parto
	return df[['biênio', 'tipo_parto', 'origem', 'destino', 'procedimentos']]

In [52]:
def adjust_df(fname, bienio, tipo_parto):
	df = pd.read_csv(edges_rj + fname)
	df = df.rename(columns={
		'res_codigo_adotado' : 'origem',
		'int_codigo_adotado' : 'destino',
	})
	df = df[df['origem'] == df['destino']]
	df['biênio'] = bienio
	df['tipo_parto'] = tipo_parto
	df['distancia'] = 0
	df['tempo'] = 0
	return df[['biênio', 'tipo_parto', 'origem', 'destino', 'procedimentos', 'distancia', 'tempo']]

In [49]:
df = pd.concat([
	modify_df(ces_10, '2010/11', 'cesárea'),
	modify_df(nor_10, '2010/11', 'normal'),
	modify_df(ces_18, '2018/19', 'cesárea'),
	modify_df(nor_18, '2018/19', 'normal'),
])
df

,biênio,tipo_parto,origem,destino,procedimentos
0,2010/11,cesárea,330350,330045,1207
1,2010/11,cesárea,330455,330045,798
2,2010/11,cesárea,330510,330045,380
3,2010/11,cesárea,330350,330414,362
4,2010/11,cesárea,330455,330510,310
...,...,...,...,...,...
720,2018/19,normal,330620,330610,1
721,2018/19,normal,330630,270915,1
722,2018/19,normal,330630,330411,1
723,2018/19,normal,330630,330455,1


In [54]:
df_same_mun = pd.concat([
	adjust_df(ces_10_all, '2010/11', 'cesárea'),
	adjust_df(nor_10_all, '2010/11', 'normal'),
	adjust_df(ces_18_all, '2018/19', 'cesárea'),
	adjust_df(nor_18_all, '2018/19', 'normal'),
])
df_same_mun

,biênio,tipo_parto,origem,destino,procedimentos,distancia,tempo
0,2010/11,cesárea,330455,330455,10025,0,0
1,2010/11,cesárea,330490,330490,3040,0,0
2,2010/11,cesárea,330170,330170,2005,0,0
3,2010/11,cesárea,330045,330045,1553,0,0
4,2010/11,cesárea,330100,330100,1526,0,0
...,...,...,...,...,...,...,...
196,2018/19,normal,330015,330015,10,0,0
241,2018/19,normal,330590,330590,6,0,0
273,2018/19,normal,330210,330210,4,0,0
345,2018/19,normal,330045,330045,2,0,0


In [50]:
df_res = pd.merge(
	df, df_dist,
	how='left',
	left_on = ['origem', 'destino'],
	right_on = ['origem', 'destino']
)
df_res

,biênio,tipo_parto,origem,destino,procedimentos,distancia,tempo
0,2010/11,cesárea,330350,330045,1207,7.7325,0.128667
1,2010/11,cesárea,330455,330045,798,31.2666,0.469667
2,2010/11,cesárea,330510,330045,380,6.5351,0.122250
3,2010/11,cesárea,330350,330414,362,13.8964,0.269500
4,2010/11,cesárea,330455,330510,310,25.6388,0.393500
...,...,...,...,...,...,...,...
2200,2018/19,normal,330620,330610,1,34.2078,0.591694
2201,2018/19,normal,330630,270915,1,2017.5698,25.870056
2202,2018/19,normal,330630,330411,1,35.1378,0.636833
2203,2018/19,normal,330630,330455,1,127.4334,1.937028


In [57]:
df_final = pd.concat([
	df_res,
	df_same_mun
])
df_final = df_final.sort_values('procedimentos', ascending=False)
df_final

,biênio,tipo_parto,origem,destino,procedimentos,distancia,tempo
0,2018/19,normal,330455,330455,56423,0.0000,0.000000
0,2010/11,normal,330455,330455,49736,0.0000,0.000000
0,2018/19,cesárea,330455,330455,20304,0.0000,0.000000
0,2010/11,cesárea,330455,330455,10025,0.0000,0.000000
1,2010/11,normal,330490,330490,9972,0.0000,0.000000
...,...,...,...,...,...,...,...
1263,2018/19,cesárea,330245,330330,1,184.0067,2.909417
1262,2018/19,cesárea,330240,330550,1,118.7163,1.602889
1261,2018/19,cesárea,330240,330490,1,163.6184,2.248528
1260,2018/19,cesárea,330240,330430,1,110.4914,1.583083


In [58]:
df_final.to_csv(edges_rj + 'edges_with_dist.csv', index=False)

In [5]:
df_final = pd.read_csv(edges_rj + 'edges_with_dist.csv')
df_final

,biênio,tipo_parto,origem,destino,procedimentos,distancia,tempo
0,2018/19,normal,330455,330455,56423,0.0000,0.000000
1,2010/11,normal,330455,330455,49736,0.0000,0.000000
2,2018/19,cesárea,330455,330455,20304,0.0000,0.000000
3,2010/11,cesárea,330455,330455,10025,0.0000,0.000000
4,2010/11,normal,330490,330490,9972,0.0000,0.000000
...,...,...,...,...,...,...,...
2481,2018/19,cesárea,330245,330330,1,184.0067,2.909417
2482,2018/19,cesárea,330240,330550,1,118.7163,1.602889
2483,2018/19,cesárea,330240,330490,1,163.6184,2.248528
2484,2018/19,cesárea,330240,330430,1,110.4914,1.583083


In [11]:
df_final[(df_final['origem'] == 330455) & (df_final['procedimentos'] == 6)]

,biênio,tipo_parto,origem,destino,procedimentos,distancia,tempo
890,2018/19,normal,330455,220840,6,2775.4889,34.853667
908,2018/19,cesárea,330455,211010,6,2884.1629,36.885444
910,2018/19,normal,330455,211250,6,3025.2206,38.369611
911,2018/19,normal,330455,250690,6,2420.2223,30.911306


In [7]:
pth = '/Users/lucas/Desktop/doc_topics.xlsx'
ex = pd.read_excel(pth, sheet_name=1)
ex

ValueError: Worksheet index 1 is invalid, 1 worksheets found

In [5]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39052 entries, 0 to 39051
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unnamed: 0                               39052 non-null  int64  
 1   id                                       39052 non-null  int64  
 2   date                                     39052 non-null  object 
 3   period                                   39052 non-null  int64  
 4   type                                     39052 non-null  object 
 5   n_authors                                39052 non-null  int64  
 6   hic                                      39052 non-null  object 
 7   lmic                                     39038 non-null  object 
 8   producer                                 39052 non-null  object 
 9   tags                                     39052 non-null  object 
 10  Artificial Intelligence                  39052